In [ ]:
!pip install torch torchvision
!pip install opencv-python-headless
!pip install dlib
!pip install recognition
!pip install tqdm
!pip install -q torch torchvision
!pip install -q opencv-python-headless dlib face_recognition
!pip install face_recognition
!pip install dlib



ERROR: Could not find a version that satisfies the requirement recognition (from versions: none)
ERROR: No matching distribution found for recognition


In [ ]:
# Veri indirme
from google.colab import files
files.upload()

# Google Drive'ı bağla
from google.colab import drive
drive.mount('/content/drive')

# Gerekli kütüphaneleri kurulumu
!pip install -q kaggle kagglehub

# Kaggle API anahtarını tekrar doğru yere taşıma
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/json

# Google Drive'da verileri koyacağımız klasörü oluşturalım (varsa hata vermez)
!mkdir -p /content/drive/MyDrive/Deepfake-Detection-Project/datasets

# VERİYİ İNDİR VE DOĞRUDAN GOOGLE DRIVE'A YÜKLE
import kagglehub

# Veri setini tekrar Colab'ın geçici hafızasına indirelim
print("KaggleHub ile veri seti indirme işlemi başladı")
path = kagglehub.dataset_download("xdxd003/ff-c23")
print(f"Veri seti başarıyla şu geçici yola indirildi: {path}")

# ZIP dosyasını BU SEFER DOĞRUDAN Google Drive'daki hedef klasörümüze çıkartalım
print("ZIP dosyası DOĞRUDAN Google Drive'a çıkartılıyor...")

!unzip -q "{path}"/*.zip -d /content/drive/MyDrive/Deepfake-Detection-Project/datasets/

print("Veri seti başarıyla Google Drive'a çıkartıldı!")

In [ ]:
# VİZE İÇİN BASE MODEL VERİ İŞLEME (250 REAL + 250 FAKE)

import os
import cv2
import face_recognition
from google.colab import drive
from tqdm.notebook import tqdm
import glob
import random

# Drive'ı bağla ve yolları tanımla
drive.mount('/content/drive')

DRIVE_BASE_PATH = "/content/drive/MyDrive/Deepfake-Detection-Project"
RAW_DATA_PATH = os.path.join(DRIVE_BASE_PATH, "datasets", "FaceForensics++_C23")
# Çıktı klasörü
OUTPUT_PATH = os.path.join(DRIVE_BASE_PATH, "processed_data_500_videos")

NUM_FRAMES_PER_VIDEO = 20 # Her videodan 20 kare alalım
IMAGE_SIZE = 224

# Çıktı klasörlerini oluşturma
os.makedirs(os.path.join(OUTPUT_PATH, "real"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_PATH, "fake"), exist_ok=True)

print(f"Ham video veriler: {RAW_DATA_PATH}")
print(f"İşlenmiş yüz resimleri buraya kaydedilecek: {OUTPUT_PATH}")

def process_videos_final(video_paths, output_dir_label):

    output_path = os.path.join(OUTPUT_PATH, output_dir_label)

    for video_path in tqdm(video_paths, desc=f"Processing {output_dir_label} videos"):
        video_name = os.path.basename(video_path).split('.')[0]
        try:
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened(): continue
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            if total_frames < 1: continue

            frame_indices = [int(i) for i in (total_frames / NUM_FRAMES_PER_VIDEO * (j + 0.5) for j in range(NUM_FRAMES_PER_VIDEO))] if total_frames > NUM_FRAMES_PER_VIDEO else list(range(total_frames))

            frame_count = 0
            saved_frames = 0
            while saved_frames < len(frame_indices):
                ret, frame = cap.read()
                if not ret: break
                if frame_count in frame_indices:
                    face_locations = face_recognition.face_locations(frame)
                    if len(face_locations) > 0:
                        top, right, bottom, left = face_locations[0]
                        face_image = frame[top:bottom, left:right]
                        if face_image.size > 0:
                            resized_face = cv2.resize(face_image, (IMAGE_SIZE, IMAGE_SIZE))
                            save_name = f"{video_name}_frame_{frame_count}.jpg"
                            cv2.imwrite(os.path.join(output_path, save_name), resized_face)
                            saved_frames += 1
                frame_count += 1
            cap.release()
        except Exception as e:
            print(f"Bir hata oluştu: {video_name} - {e}")
            continue

# VERİ SEÇİMİ VE OPERASYON BAŞLANGICI

# Adım 1: 250 Gerçek video seçimi
real_videos = sorted(glob.glob(os.path.join(RAW_DATA_PATH, "original", "*.mp4")))[:250]

# Adım 2: Her bir fake klasöründen 50'şer video seçimi
fake_video_folders = ["Deepfakes", "Face2Face", "FaceShifter", "FaceSwap", "NeuralTextures"]
fake_videos = []
for folder in fake_video_folders:
    videos_in_folder = sorted(glob.glob(os.path.join(RAW_DATA_PATH, folder, "*.mp4")))
    fake_videos.extend(videos_in_folder[:50])

print(f"İşlemek için {len(real_videos)} gerçek video seçildi.")
print(f"İşlemek için {len(fake_videos)} sahte video seçildi (her bir 5 fake dosyasından 50'şer adet).")

# Operasyon
process_videos_final(real_videos, "real")
process_videos_final(fake_videos, "fake")

print("\n\n BASE MODEL İÇİN VERİ İŞLEME TAMAMLANDI!")
print(f"Tüm yüz resimleri Google Drive'daki '{OUTPUT_PATH}' klasörüne kaydedildi.")

In [ ]:
# Model eğitimi


# BÖLÜM 1: Kütüphaneler ve Ayarlar
import torch, torch.nn as nn, torch.optim as optim, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os, glob
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DRIVE_BASE_PATH = "/content/drive/MyDrive/Deepfake-Detection-Project"
DATA_PATH = os.path.join(DRIVE_BASE_PATH, "processed_data_500_videos")
CHECKPOINT_PATH = os.path.join(DRIVE_BASE_PATH, "checkpoints") # Kayıt klasörü
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

LEARNING_RATE = 0.001
BATCH_SIZE = 32
NUM_EPOCHS = 15
IMAGE_SIZE = 224

# BÖLÜM 2: Dataset Sınıfı (Aynı)
class DeepfakeDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths, self.labels, self.transform = image_paths, labels, transform
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = torch.tensor([self.labels[idx]], dtype=torch.float32)
        if self.transform:
            image = self.transform(image)
        return image, label

# BÖLÜM 3: Sıfırdan Basit CNN Modelimiz (Aynı)
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1, self.pool1 = nn.Conv2d(3, 16, 3, padding=1), nn.MaxPool2d(2, 2)
        self.conv2, self.pool2 = nn.Conv2d(16, 32, 3, padding=1), nn.MaxPool2d(2, 2)
        self.conv3, self.pool3 = nn.Conv2d(32, 64, 3, padding=1), nn.MaxPool2d(2, 2)
        self.fc1, self.fc2 = nn.Linear(64 * 28 * 28, 512), nn.Linear(512, 1)
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# BÖLÜM 4: Veri Hazırlama ve Yükleyiciler (Aynı)
all_images = glob.glob(os.path.join(DATA_PATH, "real", "*.jpg")) + glob.glob(os.path.join(DATA_PATH, "fake", "*.jpg"))
all_labels = [0] * len(glob.glob(os.path.join(DATA_PATH, "real", "*.jpg"))) + [1] * len(glob.glob(os.path.join(DATA_PATH, "fake", "*.jpg")))
train_paths, val_paths, train_labels, val_labels = train_test_split(all_images, all_labels, test_size=0.2, random_state=42, stratify=all_labels)
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
train_dataset, val_dataset = DeepfakeDataset(train_paths, train_labels, transform=transform), DeepfakeDataset(val_paths, val_labels, transform=transform)
train_loader, val_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True), DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# BÖLÜM 5: Model, Kayıp Fonksiyonu ve Optimizatör (Aynı)
model = SimpleCNN().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# BÖLÜM 6: Eğitim Döngüsü (GÜNCELLENDİ)
best_accuracy = 0.0

for epoch in range(NUM_EPOCHS):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    corrects = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.sigmoid(outputs) > 0.5
            corrects += torch.sum(preds == labels.data)
    val_accuracy = (corrects.double() / len(val_dataset)) * 100
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} -> Val Accuracy: {val_accuracy:.2f}%")

    #Modeli Kaydet
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        BEST_MODEL_PATH = os.path.join(CHECKPOINT_PATH, "simple_cnn_best_vize_modeli.pth")
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print(f" Yeni en iyi model bulundu ve '{BEST_MODEL_PATH}' dosyasına kaydedildi!")

print("\n\nMODEL EĞİTİMİ TAMAMLANDI")

Mounted at /content/drive
Epoch 1/15 -> Val Accuracy: 73.76%
 Yeni en iyi model bulundu ve '/content/drive/MyDrive/Deepfake_Projesi/checkpoints/simple_cnn_best_vize_modeli.pth' dosyasına kaydedildi!
Epoch 2/15 -> Val Accuracy: 76.49%
 Yeni en iyi model bulundu ve '/content/drive/MyDrive/Deepfake_Projesi/checkpoints/simple_cnn_best_vize_modeli.pth' dosyasına kaydedildi!
Epoch 3/15 -> Val Accuracy: 81.86%
 Yeni en iyi model bulundu ve '/content/drive/MyDrive/Deepfake_Projesi/checkpoints/simple_cnn_best_vize_modeli.pth' dosyasına kaydedildi!
Epoch 4/15 -> Val Accuracy: 83.44%
 Yeni en iyi model bulundu ve '/content/drive/MyDrive/Deepfake_Projesi/checkpoints/simple_cnn_best_vize_modeli.pth' dosyasına kaydedildi!
Epoch 5/15 -> Val Accuracy: 82.58%
Epoch 6/15 -> Val Accuracy: 82.51%
Epoch 7/15 -> Val Accuracy: 81.86%
Epoch 8/15 -> Val Accuracy: 84.73%
 Yeni en iyi model bulundu ve '/content/drive/MyDrive/Deepfake_Projesi/checkpoints/simple_cnn_best_vize_modeli.pth' dosyasına kaydedildi!
Epoc

In [ ]:
import os
import glob
import shutil

# Proje Yolları
DRIVE_BASE_PATH = "/content/drive/MyDrive/Deepfake-Detection-Project"
RAW_DATA_PATH = os.path.join(DRIVE_BASE_PATH, "datasets", "FaceForensics++_C23")
TEST_DATA_PATH = os.path.join(DRIVE_BASE_PATH, "final_test_data_25_25") # Anlaşılır olması için yeni bir isim verdim

# Önceki test verisi varsa diye temizleyelim
if os.path.exists(TEST_DATA_PATH):
    shutil.rmtree(TEST_DATA_PATH)
    print(f"Eski '{TEST_DATA_PATH}' klasörü temizlendi.")

# Test klasörlerini yeniden oluşturalım
os.makedirs(os.path.join(TEST_DATA_PATH, "real"), exist_ok=True)
os.makedirs(os.path.join(TEST_DATA_PATH, "fake"), exist_ok=True)
print(f"Yeni '{TEST_DATA_PATH}' klasörü oluşturuldu.")

# ==============================================================================
# 1. 25 ADET GERÇEK VİDEO AL
# ==============================================================================
# Eğitimde ilk 250'yi kullanmıştık. Şimdi 250'den sonraki 25 videoyu alıyoruz.
real_videos_to_test = sorted(glob.glob(os.path.join(RAW_DATA_PATH, "original", "*.mp4")))[250:275]

for video in real_videos_to_test:
    shutil.copy(video, os.path.join(TEST_DATA_PATH, "real"))

print(f"-> {len(real_videos_to_test)} adet GERÇEK video 'real' klasörüne başarıyla kopyalandı.")


# ==============================================================================
# 2. 25 ADET SAHTE VİDEO AL (SADECE DEEPFAKES KLASÖRÜNDEN)
# ==============================================================================
# Eğitimde ilk 50'yi kullanmıştık. Şimdi 50'den sonraki 25 videoyu alıyoruz.
fake_videos_to_test = sorted(glob.glob(os.path.join(RAW_DATA_PATH, "Deepfakes", "*.mp4")))[50:75]

for video in fake_videos_to_test:
    shutil.copy(video, os.path.join(TEST_DATA_PATH, "fake"))

print(f"-> {len(fake_videos_to_test)} adet SAHTE video 'fake' klasörüne başarıyla kopyalandı.")

print("\nİstediğin gibi, 25 gerçek ve 25 sahte videodan oluşan test seti hazır!")

Eski '/content/drive/MyDrive/Deepfake_Projesi/final_test_data_25_25' klasörü temizlendi.
Yeni '/content/drive/MyDrive/Deepfake_Projesi/final_test_data_25_25' klasörü oluşturuldu.
-> 25 adet GERÇEK video 'real' klasörüne başarıyla kopyalandı.
-> 25 adet SAHTE video 'fake' klasörüne başarıyla kopyalandı.

İstediğin gibi, 25 gerçek ve 25 sahte videodan oluşan test seti hazır!


In [ ]:
# BÖLÜM 9: VİZE İÇİN BASE MODEL FİNAL TESTİ

import torch, torch.nn as nn, torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import os, glob, cv2, face_recognition
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# AYARLAR
# Test edilecek modelin '.pth' dosyasının yolu
MODEL_PATH = "/content/drive/MyDrive/Deepfake-Detection-Project/checkpoints/simple_cnn_best_vize_modeli.pth"

# Gerekli tanımlamalar
DRIVE_BASE_PATH = "/content/drive/MyDrive/Deepfake-Detection-Project"
TEST_DATA_PATH = os.path.join(DRIVE_BASE_PATH, "final_test_data_25_25")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_INPUT_SIZE = 224

# Test edilecek modelin mimarisi
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1, self.pool1 = nn.Conv2d(3, 16, 3, padding=1), nn.MaxPool2d(2, 2)
        self.conv2, self.pool2 = nn.Conv2d(16, 32, 3, padding=1), nn.MaxPool2d(2, 2)
        self.conv3, self.pool3 = nn.Conv2d(32, 64, 3, padding=1), nn.MaxPool2d(2, 2)
        self.fc1, self.fc2 = nn.Linear(64 * 28 * 28, 512), nn.Linear(512, 1)
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x))); x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x))); x = x.view(-1, 64 * 28 * 28)
        x = F.relu(self.fc1(x)); x = self.dropout(x); x = self.fc2(x)
        return x

#Model Yükleme
print("Basit CNN modeli yükleniyor...")
model = SimpleCNN().to(device)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval() # Modeli çıkarım moduna al
print("Model başarıyla yüklendi. Test başlıyor...")

# Video Tahmin Fonksiyonu
def predict_video(video_path, model):
    transform = transforms.Compose([
        transforms.Resize((MODEL_INPUT_SIZE, MODEL_INPUT_SIZE)), transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened(): return "HATA"
    face_predictions = []
    frame_count = 0
    while frame_count < 30:
        ret, frame = cap.read()
        if not ret: break
        face_locations = face_recognition.face_locations(frame)
        if len(face_locations) > 0:
            top, right, bottom, left = face_locations[0]
            face_image = frame[top:bottom, left:right]
            if face_image.size > 0:
                pil_image = Image.fromarray(cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB))
                input_tensor = transform(pil_image).unsqueeze(0).to(device)
                with torch.no_grad():
                    output = model(input_tensor)
                    prob = torch.sigmoid(output).item()
                    face_predictions.append(1 if prob > 0.5 else 0)
        frame_count += 1
    cap.release()
    if not face_predictions: return "Yüz Bulunamadı"
    final_prediction = 1 if sum(face_predictions) > len(face_predictions) / 2 else 0
    return "FAKE" if final_prediction == 1 else "REAL"

#Ana Test Döngüsü
all_videos = glob.glob(os.path.join(TEST_DATA_PATH, "**", "*.mp4"), recursive=True)
true_labels, predicted_labels = [], []
for video_path in tqdm(all_videos, desc="Test Videoları Değerlendiriliyor"):
    ground_truth = "REAL" if "real" in video_path else "FAKE"
    prediction = predict_video(video_path, model)
    if prediction in ["REAL", "FAKE"]:
        true_labels.append(ground_truth)
        predicted_labels.append(prediction)

#Sonuçları Hesaplama ve Gösterme
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, pos_label='FAKE')
recall = recall_score(true_labels, predicted_labels, pos_label='FAKE')
f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
conf_matrix = confusion_matrix(true_labels, predicted_labels, labels=["REAL", "FAKE"])

print("\n\n" + "="*30); print(f"PERFORMANS ANALİZİ SONUÇLARI (SimpleCNN)"); print("="*30)
print(f"Toplam Test Edilen Video Sayısı: {len(true_labels)}")
print(f"Doğruluk (Accuracy): {accuracy * 100:.2f}%")
print(f"Hassasiyet (Precision for FAKE): {precision * 100:.2f}%")
print(f"Duyarlılık (Recall for FAKE): {recall * 100:.2f}%")
print(f"F1 Skoru (F1 Score for FAKE): {f1:.2f}")
print("\nKarışıklık Matrisi (Confusion Matrix):"); print("          Tahmin: REAL | Tahmin: FAKE")
print(f"Gerçek: REAL | {conf_matrix[0][0]:<11} | {conf_matrix[0][1]}")
print(f"Gerçek: FAKE | {conf_matrix[1][0]:<11} | {conf_matrix[1][1]}"); print("="*30)

Basit CNN modeli yükleniyor...
Model başarıyla yüklendi. Test başlıyor...


Test Videoları Değerlendiriliyor:   0%|          | 0/50 [00:00<?, ?it/s]



PERFORMANS ANALİZİ SONUÇLARI (SimpleCNN)
Toplam Test Edilen Video Sayısı: 50
Doğruluk (Accuracy): 58.00%
Hassasiyet (Precision for FAKE): 83.33%
Duyarlılık (Recall for FAKE): 20.00%
F1 Skoru (F1 Score for FAKE): 0.32

Karışıklık Matrisi (Confusion Matrix):
          Tahmin: REAL | Tahmin: FAKE
Gerçek: REAL | 24          | 1
Gerçek: FAKE | 20          | 5
